<a href="https://colab.research.google.com/github/ajits-github/NLP_with_spacy/blob/main/NLP_with_spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
nlp



---



###The Doc Object for Processed Text

In [ ]:
introduction_doc = nlp(
...     "This tutorial is about Natural Language Processing in spaCy."
... )
type(introduction_doc)
[token.text for token in introduction_doc]

['This',
 'tutorial',
 'is',
 'about',
 'Natural',
 'Language',
 'Processing',
 'in',
 'spaCy',
 '.']

In [ ]:
import pathlib
file_name = "/content/introduction.txt"
introduction_doc = nlp(pathlib.Path(file_name).read_text(encoding="utf-8"))
print ([token.text for token in introduction_doc])

['"', 'This', 'tutorial', 'is', 'about', 'Natural', 'Language', 'Processing', 'in', 'spaCy', '.', '"']




---



###Sentence Detection

In [ ]:
about_text = (
    "Gus Proto is a Python developer currently"
    " working for a London-based Fintech"
    " company. He is interested in learning"
    " Natural Language Processing."
)
about_doc = nlp(about_text)
sentences = list(about_doc.sents)
len(sentences)

for sentence in sentences:
    print(f"{sentence[:5]}...")

Gus Proto is a Python...
He is interested in learning...


In [ ]:
ellipsis_text = (
    "Gus, can you, ... never mind, I forgot"
    " what I was saying. So, do you think"
    " we should ..."
)

from spacy.language import Language
@Language.component("set_custom_boundaries")
def set_custom_boundaries(doc):
    """Add support to use `...` as a delimiter for sentence detection"""
    for token in doc[:-1]:
        if token.text == "...":
            doc[token.i + 1].is_sent_start = True
    return doc


custom_nlp = spacy.load("en_core_web_sm")
custom_nlp.add_pipe("set_custom_boundaries", before="parser")
custom_ellipsis_doc = custom_nlp(ellipsis_text)
custom_ellipsis_sentences = list(custom_ellipsis_doc.sents)
for sentence in custom_ellipsis_sentences:
    print(sentence)

Gus, can you, ...
never mind, I forgot what I was saying.
So, do you think we should ...


* We used the @Language.component("set_custom_boundaries") decorator to define a new function that takes a Doc object as an argument. The job of this function is to identify tokens in Doc that are the beginning of sentences and mark their .is_sent_start attribute to True. Once done, the function must return the Doc object again



---



###Tokens in spaCy

Building the Doc container involves tokenizing the text. The process of tokenization breaks a text down into its basic units—or tokens—which are represented in spaCy as Token objects.

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
about_text = (
    "Gus Proto is a Python developer currently"
    " working for a London-based Fintech"
    " company. He is interested in learning"
    " Natural Language Processing."
)
about_doc = nlp(about_text)

for token in about_doc:
    print (token, token.idx)

Gus 0
Proto 4
is 10
a 13
Python 15
developer 22
currently 32
working 42
for 50
a 54
London 56
- 62
based 63
Fintech 69
company 77
. 84
He 86
is 89
interested 92
in 103
learning 106
Natural 115
Language 123
Processing 132
. 142


In [ ]:
print(f'{"Text with Whitespace":22}'
      f'{"Is Alphanumeric?":15}'
      f'{"Is Punctuation?":18}'
      f'{"Is Stop Word?"}'
)

for token in about_doc:
    print(
        f"{str(token.text_with_ws):22}"
        f"{str(token.is_alpha):15}"
        f"{str(token.is_punct):18}"
        f"{str(token.is_stop)}"
    )

Text with Whitespace  Is Alphanumeric?Is Punctuation?   Is Stop Word?
Gus                   True           False             False
Proto                 True           False             False
is                    True           False             True
a                     True           False             True
Python                True           False             False
developer             True           False             False
currently             True           False             False
working               True           False             False
for                   True           False             True
a                     True           False             True
London                True           False             False
-                     False          True              False
based                 True           False             False
Fintech               True           False             False
company               True           False             False
.                  

* text_with_ws prints the token text along with any trailing space, if present
* is_alpha indicates whether the token consists of alphabetic characters or not
* is_punct indicates whether the token is a punctuation symbol or not
* is_stop indicates whether the token is a stop word or not

In [ ]:
custom_about_text = (
    "Gus Proto is a Python developer currently"
    " working for a London@based Fintech"
    " company. He is interested in learning"
    " Natural Language Processing."
)

print([token.text for token in nlp(custom_about_text)[8:15]])

['for', 'a', 'London@based', 'Fintech', 'company', '.', 'He']


* As with many aspects of spaCy, we can also customize the tokenization process to detect tokens on custom characters. This is often used for hyphenated words such as London-based.

* To customize tokenization, you need to update the tokenizer property on the callable Language object with a new Tokenizer object.

* To see what's involved, imagine you had some text that used the @ symbol instead of the usual hyphen (-) as an infix to link words together. So, instead of London-based, we had London@based

* In this example, the default parsing read the London@based text as a single token, but if we had used a hyphen instead of the @ symbol, then we'd get three tokens.

* To include the @ symbol as a custom infix, we need to build your own Tokenizer object

In [ ]:
import re
from spacy.tokenizer import Tokenizer

custom_nlp = spacy.load("en_core_web_sm")
prefix_re = spacy.util.compile_prefix_regex(
    custom_nlp.Defaults.prefixes
)
suffix_re = spacy.util.compile_suffix_regex(
    custom_nlp.Defaults.suffixes
)

custom_infixes = [r"@"]

infix_re = spacy.util.compile_infix_regex(
    list(custom_nlp.Defaults.infixes) + custom_infixes
)

custom_nlp.tokenizer = Tokenizer(
    nlp.vocab,
    prefix_search=prefix_re.search,
    suffix_search=suffix_re.search,
    infix_finditer=infix_re.finditer,
    token_match=None,
)

custom_tokenizer_about_doc = custom_nlp(custom_about_text)

print([token.text for token in custom_tokenizer_about_doc[8:15]])

['for', 'a', 'London', '@', 'based', 'Fintech', 'company']


###Stop Words

Stop words are typically defined as the most common words in a language. In the English language, some examples of stop words are the, are, but, and they. Most sentences need to contain stop words in order to be full sentences that make grammatical sense.

With NLP, stop words are generally removed because they aren’t significant, and they heavily distort any word frequency analysis. spaCy stores a list of stop words for the English language

In [ ]:
import spacy
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
len(spacy_stopwords)

for stop_word in list(spacy_stopwords)[:10]:
    print(stop_word)

across
've
hers
beforehand
re
six
namely
is
’ve
we


In [ ]:
custom_about_text = (
    "Gus Proto is a Python developer currently"
    " working for a London-based Fintech"
    " company. He is interested in learning"
    " Natural Language Processing."
)
nlp = spacy.load("en_core_web_sm")
about_doc = nlp(custom_about_text)
print([token for token in about_doc if not token.is_stop])

[Gus, Proto, Python, developer, currently, working, London, -, based, Fintech, company, ., interested, learning, Natural, Language, Processing, .]


###Lemmatization

Lemmatization is the process of reducing inflected forms of a word while still ensuring that the reduced form belongs to the language. This reduced form, or root word, is called a lemma.

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
conference_help_text = (
    "Gus is helping organize a developer"
    " conference on Applications of Natural Language"
    " Processing. He keeps organizing local Python meetups"
    " and several internal talks at his workplace."
)
conference_help_doc = nlp(conference_help_text)
for token in conference_help_doc:
    if str(token) != str(token.lemma_):
        print(f"{str(token):>20} : {str(token.lemma_)}")

###Word Embeddings

Word embedding techniques - Word2Vec, GloVe, and FastText using Gensim library in Python.

In [ ]:
from gensim.models import Word2Vec
sentences = [['I', 'love', 'machine', 'learning'],
             ['Word', 'embeddings', 'are', 'powerful'],
             ['Natural', 'language', 'processing', 'is', 'interesting']]

model_w2v = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
word_vector = model_w2v.wv['learning']
print(word_vector)


[-0.00515624 -0.00666834 -0.00777684  0.00831073 -0.00198234 -0.00685496
 -0.00415439  0.00514413 -0.00286914 -0.00374966  0.00162143 -0.00277629
 -0.00158436  0.00107449 -0.00297794  0.00851928  0.00391094 -0.00995886
  0.0062596  -0.00675425  0.00076943  0.00440423 -0.00510337 -0.00211067
  0.00809548 -0.00424379 -0.00763626  0.00925791 -0.0021555  -0.00471943
  0.0085708   0.00428334  0.00432484  0.00928451 -0.00845308  0.00525532
  0.00203935  0.00418828  0.0016979   0.00446413  0.00448629  0.00610452
 -0.0032021  -0.00457573 -0.00042652  0.00253373 -0.00326317  0.00605772
  0.00415413  0.00776459  0.00256927  0.00811668 -0.00138721  0.00807793
  0.00371702 -0.00804732 -0.00393361 -0.00247188  0.00489304 -0.00087216
 -0.00283091  0.00783371  0.0093229  -0.00161493 -0.00515925 -0.00470176
 -0.00484605 -0.00960283  0.00137202 -0.00422492  0.00252671  0.00561448
 -0.00406591 -0.00959658  0.0015467  -0.00670012  0.00249517 -0.00378063
  0.00707842  0.00064022  0.00356094 -0.00273913 -0

In [ ]:
from gensim.models import FastText
sentences = [['I', 'love', 'machine', 'learning'],
             ['Word', 'embeddings', 'are', 'powerful'],
             ['Natural', 'language', 'processing', 'is', 'interesting']]

model_fasttext = FastText(sentences, vector_size=100, window=5, min_count=1, workers=4)
word_vector = model_fasttext.wv['learning']
print(word_vector)


[ 9.1823016e-04 -3.0900480e-04 -1.4241622e-03  3.6915974e-04
 -1.6989922e-03 -4.7368856e-04 -9.6330949e-04 -8.1712339e-04
  1.4180044e-03  3.7115382e-04 -1.1983200e-04  2.9081755e-04
 -9.9326216e-04  1.0794357e-03  5.1063707e-04 -1.1530268e-03
 -1.5505445e-03 -1.9562184e-03  5.7489559e-04 -2.0934530e-03
 -1.8944317e-03 -2.0447427e-03  1.0821981e-03 -2.2537062e-04
 -5.0764071e-04 -1.0467614e-03 -6.9561160e-05 -7.7934202e-04
 -3.2007642e-04 -1.4727070e-03 -1.4016990e-03  1.8091477e-03
  7.5201882e-04  4.1703676e-04 -1.6488129e-04  1.2592709e-03
 -4.0854918e-04  5.9077330e-04 -1.3326405e-03 -8.8910357e-04
 -1.0974610e-03 -1.0485642e-03 -7.7783951e-04  6.8859063e-04
 -1.1026027e-03  3.0985163e-04 -8.7897031e-05  4.3270687e-04
  2.0680524e-04  7.2805583e-04  1.5761263e-03 -4.8463579e-04
 -3.9394345e-04  1.9359115e-03  9.9428324e-04  7.9648633e-04
 -2.5278129e-04  2.9308398e-04 -6.4702188e-05 -9.9841051e-04
  3.2134666e-04 -6.8074651e-04 -1.5585172e-03  1.4566220e-03
 -1.5276216e-03 -1.03553

FastText with spacy:

In [ ]:
from gensim.models import FastText
import spacy

# Sample sentences
sentences = ['I love machine learning.',
             'Word embeddings are powerful.',
             'Natural language processing is interesting.']

# Tokenize and preprocess the sentences using spacy
nlp = spacy.load('en_core_web_sm')
preprocessed_sentences = [[token.text.lower() for token in nlp(sentence) if token.is_alpha] for sentence in sentences]

# Train FastText model
model_fasttext = FastText(sentences=preprocessed_sentences, vector_size=100, window=5, min_count=1, workers=4)

# Get word vector for 'learning'
word_vector = model_fasttext.wv['learning']
print(word_vector)


[ 9.1823016e-04 -3.0900480e-04 -1.4241622e-03  3.6915974e-04
 -1.6989922e-03 -4.7368856e-04 -9.6330949e-04 -8.1712339e-04
  1.4180044e-03  3.7115382e-04 -1.1983200e-04  2.9081755e-04
 -9.9326216e-04  1.0794357e-03  5.1063707e-04 -1.1530268e-03
 -1.5505445e-03 -1.9562184e-03  5.7489559e-04 -2.0934530e-03
 -1.8944317e-03 -2.0447427e-03  1.0821981e-03 -2.2537062e-04
 -5.0764071e-04 -1.0467614e-03 -6.9561160e-05 -7.7934202e-04
 -3.2007642e-04 -1.4727070e-03 -1.4016990e-03  1.8091477e-03
  7.5201882e-04  4.1703676e-04 -1.6488129e-04  1.2592709e-03
 -4.0854918e-04  5.9077330e-04 -1.3326405e-03 -8.8910357e-04
 -1.0974610e-03 -1.0485642e-03 -7.7783951e-04  6.8859063e-04
 -1.1026027e-03  3.0985163e-04 -8.7897031e-05  4.3270687e-04
  2.0680524e-04  7.2805583e-04  1.5761263e-03 -4.8463579e-04
 -3.9394345e-04  1.9359115e-03  9.9428324e-04  7.9648633e-04
 -2.5278129e-04  2.9308398e-04 -6.4702188e-05 -9.9841051e-04
  3.2134666e-04 -6.8074651e-04 -1.5585172e-03  1.4566220e-03
 -1.5276216e-03 -1.03553